(vis-tables)=
# Tables


## Introduction

The humble table is an underloved and underappreciated element of communicating analysis. While it may not be as visually engaging as a vivid graph (and is far less good for a general audience), it has the advantage of being able to convey exact numerical information. It's also an essential part of some analysis: for example, when writing economics papers, there is usually a "table 1" that contains descriptive statistics. (For more on best practice for tables, check out the advice provided by the UK government's [Analysis Function](https://analysisfunction.civilservice.gov.uk/policy-store/data-visualisation-tables/).)

### Options

There are three main options for tables in Python.

1. **pandas**, which has some solid table export options. This is especially convenient if your data are already in a dataframe.
2. The **gt** (**g**reat **t**ables) package by Posit, who also made an R version of the same package. This makes really good HTML and latex tables: [check here for the package website](https://posit-dev.github.io/great-tables/articles/intro.html).
3. Finally, if you're happy with *image* output formats, **matplotlib**—the infinitely flexible image package—is great. However, your tables *have* to be exported as image files rather than as machine-readable text-based files. If you want a *graphic* table that is more striking, **matplotib** is brilliant. I wouldn't say it's the simplest way to create tables, as it relies on content featured in {ref}`vis-matplotlib` and {ref}`vis-common-plots-one`, plus more besides. Rather than cover it here, instead we're just going to direct you to this excellent [blog post](https://karbartolome.quarto.pub/the-grammar-of-tables/) on 'the grammar of tables' by Karina Bartolomé that walks through creating a table with **matplotlib**.

Note that some packages, like **pyfixest**, come with their own options to export tables. In the case of **Pyfixest**, there's a built-in way to export regression tables to formats including latex. You can find more on this in [the page on regressions](/econmt-regression.ipynb).

## **pandas** for tables

### Imports and setup

As ever, we'll start by importing some key packages and initialising any settings:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Set seed for random numbers
seed_for_prng = 78557
prng = np.random.default_rng(
    seed_for_prng
)  # prng=probabilistic random number generator

In [ ]:
import matplotlib_inline.backend_inline

# Plot settings
plt.style.use(
    "https://github.com/aeturrell/coding-for-economists/raw/main/plot_style.txt"
)
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

# Set max rows displayed for readability
pd.set_option("display.max_rows", 10)
pd.set_option("display.max_columns", 7)

We'll use the *penguins* dataset to demonstrate the use of **pandas** in creating tables. These data come with the **seaborn** package, which you'll need:

In [ ]:
import seaborn as sns

pen = sns.load_dataset("penguins")
pen.head()

### Preparing and creating your table in **pandas**

There are a few operations that you'll want to do again, and again, and again to create tables. A cross-tab is one such operation! A cross-tab is just a count of the number of elements split by two groupings. Rather than just display this using the `pd.crosstab()` function, we can add totals or percentages using the `margins=` and `normalize=` commands.

In the below, we'll use margins and normalisation so that each row sums to 1, and the last row shows the probability mass over islands.

In [ ]:
pd.crosstab(pen["species"], pen["island"], margins=True, normalize="index")

The neat thing about the cross-tabs that come out is that they are themselves **pandas** dataframes.

Of course, the usual **pandas** functions can be used to create any table you need:

In [ ]:
pen_summary = pen.groupby(["species", "island"]).agg(
    median_bill=("bill_length_mm", "median"),
    mean_bill=("bill_length_mm", "mean"),
    std_flipper=("flipper_length_mm", "std"),
)
pen_summary

For reasons that will become apparent later, we'll replace one of these values with a missing value (`pd.NA`).

In [ ]:
pen_summary.iloc[2, 1] = pd.NA

The table we just saw, `pen_summary`, is not what you'd call publication quality. The numbers have lots of superfluous digits. The names are useful for when you're doing analysis, but might not be so obvious to someone coming to this table for the first time. So let's see what we can do to clean it up a bit.

First, those numbers. We can apply number rounding quickly using `.round()`.

In [ ]:
pen_summary.round(2)

This returns another dataframe. To change the names of the columns, you can just use one of the standard approaches:

In [ ]:
pen_sum_clean = pen_summary.rename(
    columns={
        "median_bill": "Median bill length (mm)",
        "mean_bill": "Mean bill length (mm)",
        "std_flipper": "Std. deviation of flipper length",
    }
)
pen_sum_clean

One tip is to always have a dictionary up your sleeve that maps between the short names that are convenient for coding, and the longer names that you need to make outputs clear. Then, just before you do any exporting, you can always map the short names into the long names.

### Styling a **pandas** dataframe

As well as making direct modifications to a dataframe, you can apply *styles*. These are much more versatile ways to achieve styling of a table *for some output formats, namely HTML* and, although it isn't perfect, for Latex too. (But this doesn't work for markdown outputs, and markdown doesn't support such rich formatting in any case.)

Behind the scenes, when a table is displayed on a webpage like the one you're reading right now, HTML (the language most of the internet is in) is used. Styling is a way of modifying the default HTML for showing tables so that they look nicer or better.

In the example below, you can see some of the options that are available:

- `precision` is like `.round()`
- `na_rep` sets how missing values are rendered
- `thousands` sets the separator between every thousand (for readability)
- `formatter` gives fine-grained control over the formatting of individual columns


In [ ]:
pen_styled = pen_sum_clean.style.format(
    precision=3,
    na_rep="Value missing",
    thousands=",",
    formatter={
        "Mean bill length (mm)": "{:.1f}",
        "Std. deviation of flipper length (mm)": lambda x: "{:,.0f} um".format(x * 1e3),
    },
).set_caption("This is the title")
pen_styled

If you need to add more labels to either the index or the column names, you can. It's a bit fiddly, but you can.

In [ ]:
pen_sum_extra_col_info = pen_sum_clean.copy()  # create an independent copy
pen_sum_extra_col_info.columns = [["Lengths", "Lengths", "Stds"], pen_sum_clean.columns]

pen_sum_extra_col_info_styled = (
    pen_sum_extra_col_info.style.format(
        precision=3,
        na_rep="Value missing",
        thousands=",",
        formatter={
            "Mean bill length (mm)": "{:.1f}",
            "Std. deviation of flipper length (mm)": lambda x: "{:,.0f} um".format(
                x * 1e3
            ),
        },
    )
    .set_caption("This is the title")
    .set_table_styles([{"selector": "th", "props": [("text-align", "center")]}])
)
pen_sum_extra_col_info_styled

Let's see an example of exporting this to Latex too:

In [ ]:
pen_sum_extra_col_info_styled.to_latex()

You can read more about the style functionality over at the [**pandas** Styler docs](https://pandas.pydata.org/docs/reference/api/pandas.io.formats.style.Styler.to_latex.html).

Another thing you might reasonably want to do is provide summary statistics for all columns, as the last column, or for all rows, as the last row. To do this for, say, the mean when *not* using the `pd.crosstab` function, you will need to insert a summary row into the dataframe object. To start with, you need to actually create the summary row—this is achieved with the `.mean(axis=0)` method to get the mean over rows. We cast this into being a dataframe using `pd.DataFrame` as otherwise it would just be a single column, or Series object. Then we need to give the columns a name that's better than the default "`0`", and we choose a multi-level (here, two level) column name recognising that the *index* of our original dataframe has two levels: species and island. We actually just want to put "summary" in once and we've arbitrarily chosen the first level for that. Note that multi-level indices and columns can get complicated but the essential trick to bear in mind is that you replace a list of strings with a list of tuples of strings, eg for the first column `["Summary:", ...]` becomes `[("Summary", "summary level two"), ...]`. Putting this all together gives us a dataframe with an *index* that is the same as the *columns* of the original dataframe:

In [ ]:
summary_row = pd.DataFrame(pen_sum_extra_col_info.mean(axis=0))
summary_row.columns = [("Summary:", "")]  # note our index has two levels
summary_row

Next we need to transpose the new summary row (so that its columns align with those in our original data frame) using `.T`, and concatenate the two dataframes together:

In [ ]:
pd.concat([pen_sum_extra_col_info, summary_row.T], axis=0)

Once this is done, you can apply all the usual stylings.

### Writing **pandas** tables to file

Writing **pandas** tables to file is fairly straightforward: just use one of **pandas** many, many output functions. These typically begin with `.to_` and then the output name. The most useful output formats will be:

- `to_html()`
- `to_latex()`
- `to_string()`
- `to_markdown()`

Add the filename you'd like to write to within the brackets following these method names. For example, to write a latex table it would be:

```python
pen_styled.to_latex(Path("outputs/table_one.tex"))
```

These files can then be picked up by other documents. Note that, sometimes, when exporting to Latex, the code will have "escape" characters, for example extra backslashes. In some versions of **pandas** you can turn these off with an `escape=False` keyword argument.

It's not perfect, but if you're writing a table to latex and want footnotes and source notes, you can make use of the `to_latex()` method's `caption=` keyword argument.

### Limitations to **pandas** tables

**pandas** tables have some limitations. It is not easy to include rich information such as images: while they can be included in the HTML rendition of a table on a webpage or in a Jupyter Notebook, it's far harder to export this kind of information sensibly to an output file that *isn't* HTML.

A more tricky general issue with them is that it can be hard to include all of the relevant information you'd expect in a table: they work extremely well for a typical table that is just rows and columns with equal-sized cells, but it you want to include, say, a long row (of a single cell) at the end for either footnotes or source notes, there isn't an obvious way to do it. Similarly, it's hard to make parts of a table such as the title, sub-title, and stubhead label work well in all cases.

To create a footnote or source note row, you could insert an extra row like above, but it's a very unsatisfactory work-around as your notes can only be in one of the columns (not spread across all of them) and it will lead to you losing the data types in at least one of your original columns.

However, there is another option for that...

## **g**reat **t**ables for tables

Newcomer [**great tables**](https://posit-dev.github.io/great-tables/articles/intro.html) has swept in and provided a compelling way to make really fancy tables right off the bat.

This section is indebted to the **great tables** documentation.

Let's start with a basic example using some data on islands:

In [ ]:
from great_tables.data import islands  # this is a pandas dataframe

islands.head()

Okay, now we're going to construct a table. Now, importantly, unlike **pandas**, we can add in a subtitle, and as many source notes as we like (notes that appeaer under the table).

In [ ]:
from great_tables import GT, md

islands_mini = islands.sort_values(by="size", ascending=False).head(10)

islands_table = (
    GT(islands_mini)
    .tab_header(
        title="Large Landmasses of the World",
        subtitle="The top ten largest are presented",
    )
    .tab_stub(rowname_col="name")
    .tab_source_note(
        source_note="Source: The World Almanac and Book of Facts, 1975, page 406."
    )
    .tab_source_note(
        source_note=md(
            "Reference: McNeil, D. R. (1977) *Interactive Data Analysis*. Wiley."
        )
    )
    .tab_stubhead(label="landmass")
    .fmt_integer(columns="size")
)

islands_table

The naming convention of the different parts of the table should be fairly intuitive but the diagram below will help you to navigate all of the possible options:

![The components of a (great) table](https://posit-dev.github.io/great-tables/assets/the_components_of_a_table.svg)

Our first example did not make use of spanner tables, but we can do so in a second, on air quality:

In [ ]:
from great_tables import GT, html
from great_tables.data import airquality

airquality_m = airquality.head(10).assign(Year=1973)

gt_airquality = (
    GT(airquality_m)
    .tab_header(
        title="New York Air Quality Measurements",
        subtitle="Daily measurements in New York City (May 1-10, 1973)",
    )
    .tab_spanner(label="Time", columns=["Year", "Month", "Day"])
    .tab_spanner(label="Measurement", columns=["Ozone", "Solar_R", "Wind", "Temp"])
    .cols_move_to_start(columns=["Year", "Month", "Day"])
    .cols_label(
        Ozone=html("Ozone,<br>ppbV"),
        Solar_R=html("Solar R.,<br>cal/m<sup>2</sup>"),
        Wind=html("Wind,<br>mph"),
        Temp=html("Temp,<br>&deg;F"),
    )
)

gt_airquality

One of the great features of great tables is that we can add in "nanoplots" that provide further information on the data being presented but in a visual format. Here's an example:

In [ ]:
numbers_df = pd.DataFrame(
    {
        "example": ["Row " + str(x) for x in range(1, 5)],
        "numbers": [
            {"val": [20, 23, 6, 7, 37, 23, 21, 4, 7, 16]},
            {"val": [2.3, 6.8, 9.2, 2.42, 3.5, 12.1, 5.3, 3.6, 7.2, 3.74]},
            {"val": [-12, -5, 6, 3.7, 0, 8, -7.4]},
            {"val": [2, 0, 15, 7, 8, 10, 1, 24, 17, 13, 6]},
        ],
    }
)

GT(numbers_df).fmt_nanoplot(columns="numbers")

And one with a min and max as a reference:

In [ ]:
GT(numbers_df).fmt_nanoplot(columns="numbers", reference_area=["min", "median"])

A line chart isn't the only type supported. For example there are bars too. And there are lots of formatting options. (Try hovering your mouse over the table below!)

In [ ]:
from great_tables import nanoplot_options

(
    GT(numbers_df).fmt_nanoplot(
        columns="numbers",
        plot_type="bar",
        autoscale=True,
        reference_line="min",
        reference_area=[0, "max"],
        options=nanoplot_options(
            data_bar_stroke_color="gray",
            data_bar_stroke_width=2,
            data_bar_fill_color="orange",
            data_bar_negative_stroke_color="blue",
            data_bar_negative_stroke_width=1,
            data_bar_negative_fill_color="lightblue",
            reference_line_color="pink",
            reference_area_fill_color="bisque",
            vertical_guide_stroke_color="blue",
        ),
    )
)

You can also colour cells as you like, or based on a condition (including the *type* of data.) Here's an example showing that off:

In [ ]:
import polars as pl
from great_tables.data import sza

# this is just some data prep
# that gets data in right shape
sza_pivot = (
    pl.from_pandas(sza)
    .filter((pl.col("latitude") == "20") & (pl.col("tst") <= "1200"))
    .select(pl.col("*").exclude("latitude"))
    .drop_nulls()
    .pivot(values="sza", index="month", on="tst", sort_columns=True)
)
sza_pivot.head()

In [ ]:
(
    GT(sza_pivot, rowname_col="month")
    .data_color(
        domain=[90, 0],
        palette=["rebeccapurple", "white", "orange"],
        na_color="white",
    )
    .tab_header(
        title="Solar Zenith Angles from 05:30 to 12:00",
        subtitle=html("Average monthly values at latitude of 20&deg;N."),
    )
)

All of these tables have shown up as gloriously rendered HTML, which is an expoert option via `GT.as_raw_html()`. And you can easily save them as high quality (lossless) image files too via `GT.save()`.

There is also a nascent latex export option, which is critical for research work. While some features aren't yet supported, if you need a vanilla table, it's still very useful. For example, for the islands dataset we saw earlier, we have to 'turn off' the use of markdown in one of the source notes as so (using regular text instead):

In [ ]:
simple_islands_table = (
    GT(islands_mini)
    .tab_header(
        title="Large Landmasses of the World",
        subtitle="The top ten largest are presented",
    )
    .tab_stub(rowname_col="name")
    .tab_source_note(
        source_note="Source: The World Almanac and Book of Facts, 1975, page 406."
    )
    .tab_source_note(
        source_note=(
            "Reference: McNeil, D. R. (1977) Interactive Data Analysis. Wiley."
        )
    )
    .tab_stubhead(label="landmass")
    .fmt_integer(columns="size")
)

simple_islands_table

Then to export to a table that can be used in your main latex report file via `\input{table_out.tex}` it would be:

```python
with open("table_out.tex", "w") as f:
    f.write(simple_islands_table.as_latex())
```